In [1]:
import math
from collections import namedtuple
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from copy import deepcopy
from tqdm import tqdm
from tqdm.notebook import tqdm
import random

import numba as nb
# from numba import jit, vectorize, float64

In [2]:
with open('./data/tsp_100_3', 'r') as input_data_file:
    input_data = input_data_file.read()
    
lines = input_data.split('\n')
nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    # points.append(Point(float(parts[0]), float(parts[1])))
    points.append((float(parts[0]), float(parts[1])))
points = np.array(points)

In [3]:
def length(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)


def scan_region_from(current_point, remaining_stops):
    x, y = current_point
    
    if x < y:
        x_min = np.min(remaining_stops[:, 0])
        scan_pointer = np.array([x_min, y])
    else:
        y_min = np.min(remaining_stops[:, 1])
        scan_pointer = np.array([x, y_min])

    return scan_pointer


def add_explored(current_stop, remaining_stops):
    ## Add node as explored
    
    # Find the index where the element matches in the array_of_arrays
    remove_idx = np.where(np.all(remaining_stops == current_stop, axis=1))
    
    # Remove the element using the indices
    remaining_stops = np.delete(remaining_stops, remove_idx, axis=0)
    
    return remaining_stops


def point_in_bbox(point, bbox):
    x, y = point
    x_min, y_min, x_max, y_max = bbox
    
    if x_min <= x <= x_max and y_min <= y <= y_max:
        return True
    else:
        return False


# def calculate_travel(graph, points):
    
#     indices_from = graph[:-1]
#     indices_to = graph[1:]
#     total_distance = np.sum(DISTANCE_MATRIX[indices_from, indices_to])

#     # Add the distance to return to the starting point
#     total_distance += DISTANCE_MATRIX[graph[-1]][graph[0]]

#     return total_distance


@nb.jit(nopython=True)
def calculate_travel(graph):
    n = len(graph)
    total_distance = 0.0

    for i in range(n - 1):
        total_distance += DISTANCE_MATRIX[graph[i], graph[i + 1]]

    # Add the distance to return to the starting point
    total_distance += DISTANCE_MATRIX[graph[-1], graph[0]]

    return total_distance


@nb.jit(nopython=True)
def calculate_travel_list(graph):
    n = len(graph)
    total_distance = np.empty(n)

    for i in range(n - 1):
        total_distance[i] = DISTANCE_MATRIX[graph[i], graph[i + 1]]

    # Add the distance to return to the starting point
    total_distance[-1] = DISTANCE_MATRIX[graph[-1], graph[0]]

    total_distance = np.argsort(-total_distance)
    
    return total_distance
    

def get_midpoint(point1, point2):
    return (point1+point2)//2

@nb.jit(nopython=True)
def do_2opt(point_idx1, point_idx2, path):
    
    if point_idx2 > point_idx1:
        slice_start = point_idx1
        slice_end = point_idx2
    else:
        slice_start = point_idx2
        slice_end = point_idx1

    # slice_path = path[slice_start:slice_end+1]
    # slice_path = np.flip(slice_path)
    # slice_path.reverse()

    # new_path = path[:slice_start]+slice_path+path[slice_end+1:]
    # new_path = np.hstack((path[:slice_start], slice_path, path[slice_end + 1:]))
    
    new_path = np.hstack((path[:slice_start], path[slice_start:slice_end][::-1], path[slice_end:]))
    
    return new_path


class TabuList:
    def __init__(self, tabu_size):
        self.tabu_size = tabu_size
        self.tabu_hash = set()
        self.tabu_queue = []

    def is_present(self, node):
        return node in self.tabu_hash

    def insert(self, node):
        if self.is_present(node):
            return
        self.tabu_hash.add(node)
        self.tabu_queue.append(node)
        if len(self.tabu_hash) > self.tabu_size:
            self.remove()
            
    def remove(self):
        top = self.tabu_queue.pop(0)
        self.tabu_hash.remove(top)

def add_penalty(path):
    util_nodes = {}
    
    for idx, node in enumerate(path):
        node_out = path[idx+1] if idx+1 < nodeCount else path[0]
        dist = DISTANCE_MATRIX[node][node_out]
        penalty = (1 + PENALTY_MATRIX[node, node_out])
        util = dist / (1 + penalty)
        util_nodes[(node, node_out)] = util

    max_util = max(util_nodes.values())
    viable_candidates = [(node, node_out) for (node, node_out), value in util_nodes.items() if value == max_util]

    for node, node_out in viable_candidates:
        PENALTY_MATRIX[node, node_out] += 1
        PENALTY_MATRIX[node_out, node] += 1

In [4]:
@nb.jit(nopython=True)
def length_distance(single_point, all_points):
    return np.sqrt((all_points[:, 0]-single_point[0])**2 + (all_points[:, 1]-single_point[1])**2)

In [5]:
PENALTY_MATRIX = np.zeros((nodeCount, nodeCount), dtype=int)

DISTANCE_MATRIX = np.zeros((nodeCount, nodeCount))
for i in tqdm(range(len(points))):
    DISTANCE_MATRIX[i] = length_distance(points[i], points)

AUGMENTED_DISTANCE_MATRIX = np.zeros((nodeCount, nodeCount))
for i in tqdm(range(len(points))):
    AUGMENTED_DISTANCE_MATRIX[i] = length_distance(points[i], points)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
optimal_path = None
optimal_distance = float('inf')
idx_pos = 0

for _ in tqdm(range(1)):
    
    exploring_path = np.array([], dtype=int)
    explore_points = deepcopy(points)
    max_neighbors = 1
    
    pick_next = 0
    exploring_path = np.append(exploring_path, pick_next)
    
    while exploring_path.size < nodeCount:
        
        neighbor_idx = np.argsort(DISTANCE_MATRIX[pick_next])
        
        mask = np.isin(neighbor_idx, exploring_path)
        neighbor_idx = neighbor_idx[~mask]
    
        pick_next = np.random.choice(neighbor_idx[:max_neighbors])
        
        exploring_path = np.append(exploring_path, pick_next)

    
    total_distance = calculate_travel(exploring_path)

    if total_distance < optimal_distance:
        optimal_path = deepcopy(exploring_path)
        optimal_distance = total_distance


global_optimal_path = deepcopy(optimal_path)
global_optimal_distance = deepcopy(optimal_distance)

optimal_distance

  0%|          | 0/1 [00:00<?, ?it/s]

25138.785452772318

In [7]:

# starting_path = np.array(range(nodeCount))
# random.shuffle(starting_path)
# starting_distance = calculate_travel(starting_path)

starting_path =  deepcopy(np.array(global_optimal_path))
starting_distance = deepcopy(global_optimal_distance)

In [8]:
global_optimal_path = deepcopy(starting_path)
global_optimal_distance = deepcopy(starting_distance)



starting_distance

25138.785452772318

In [9]:
iter = 0
max_iterations = 100000

init_temperature = 100
final_temperature = 1e-3
cooling_rate = 0.9

restart_counter = 0
temperature = init_temperature
global_temperature = init_temperature

search_idx = 100 #nodeCount//5

optimal_path = deepcopy(global_optimal_path)
optimal_distance = global_optimal_distance

last_known_distance = int(global_optimal_distance)

tabu_limit = nodeCount//5
tabu_list = TabuList(tabu_limit)

search_idx, tabu_limit

(100, 20)

In [10]:
for iter in tqdm(range(max_iterations)):
    
    if temperature < final_temperature:
        break
    
    for _ in range(100):
    # for alter_point in range(nodeCount):
        
        # pick a random point or pick points carefully        
        alter_point = random.choice(range(nodeCount))

        while tabu_list.is_present(alter_point):
            alter_point = random.choice(range(nodeCount))

        tabu_list.insert(alter_point)
        
        point_idx1 = np.where(optimal_path==alter_point)[0][0]
    
        # get all the neighbors to that point        
        sorted_neighbor_points = np.argsort(DISTANCE_MATRIX[alter_point])[1:search_idx]
        
        # repeatedly do 2-opt for all the neighbors
        for neighbor_idx in sorted_neighbor_points:
           
            point_idx2 = np.where(optimal_path==neighbor_idx)[0][0]
            new_path = do_2opt(point_idx1, point_idx2, optimal_path)
            new_distance = calculate_travel(new_path)
        
            delta = new_distance - optimal_distance
            acceptance_prob = np.exp(-delta/temperature)
            to_pick_prob = random.random()
            
            if delta < 0 or to_pick_prob < acceptance_prob:
                
                optimal_path = new_path.copy()
                optimal_distance = new_distance
                
                if new_distance < global_optimal_distance:
                    global_optimal_path = new_path.copy()
                    global_optimal_distance = new_distance
                    global_temperature = temperature

    
    if iter%10 == 0:
        print(restart_counter, iter, search_idx, round(global_optimal_distance, 4), round(temperature, 4), round(cooling_rate, 5))


    if round(global_optimal_distance, 1) < last_known_distance:
        last_known_distance = round(global_optimal_distance, 1)
        restart_counter = 0
    else:
        restart_counter += 1


    if restart_counter >= 100:
                
        if cooling_rate*1.001 < 0.995:
            restart_counter = 0  
            cooling_rate *= 1.001
            if global_temperature != init_temperature:
                temperature = global_temperature

        elif iter % 10 == 0 and search_idx > 10:
            restart_counter = 0
            search_idx = int(search_idx//1.1)
            if global_temperature != init_temperature:
                temperature = global_temperature
            
        temperature *= cooling_rate
   

    # if iter % 100 == 0 and search_idx > 15:
    #     search_idx -= 5
        
    # if restart_counter > 100 and temperature < 0.1:
    #     print("long due. ending loops")
    #     break

  0%|          | 0/100000 [00:00<?, ?it/s]

0 0 100 24301.3367 100 0.9
8 10 100 23603.837 100 0.9
2 20 100 23176.4119 100 0.9
12 30 100 23132.0016 100 0.9
1 40 100 23070.0539 100 0.9
0 50 100 22894.0814 100 0.9
9 60 100 22894.0814 100 0.9
19 70 100 22894.0814 100 0.9
2 80 100 22242.6466 100 0.9
12 90 100 22242.6466 100 0.9
22 100 100 22242.6466 100 0.9
32 110 100 22242.6466 100 0.9
42 120 100 22242.6466 100 0.9
52 130 100 22242.6466 100 0.9
62 140 100 22242.6466 100 0.9
72 150 100 22242.6466 100 0.9
82 160 100 22242.6466 100 0.9
92 170 100 22242.6466 100 0.9
0 180 100 22205.9559 90.09 0.9009
10 190 100 21959.8502 90.09 0.9009
9 200 100 21959.8502 90.09 0.9009
19 210 100 21959.8502 90.09 0.9009
29 220 100 21959.8502 90.09 0.9009
39 230 100 21959.8502 90.09 0.9009
49 240 100 21959.8502 90.09 0.9009
59 250 100 21959.8502 90.09 0.9009
69 260 100 21959.8502 90.09 0.9009
79 270 100 21959.8502 90.09 0.9009
89 280 100 21959.8502 90.09 0.9009
99 290 100 21959.8502 90.09 0.9009
9 300 100 21959.8502 81.2432 0.9018
19 310 100 21959.8502 81.

KeyboardInterrupt: 

In [15]:
output_data = '%.2f' % global_optimal_distance + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, global_optimal_path))

global_optimal_distance

20758.877866531268

In [16]:
with open(r'./results/tsp_100_3.txt', 'w') as file:
    # Write the Python code to the file
    file.write(output_data)

In [ ]:
# edge_x = []
# edge_y = []

# for idx, connections in tqdm(enumerate(graph_paths)):
    
#     x0, y0 = points[idx]
    
#     for connect in connections:
#         x1, y1 = points[connect]
    
#         edge_x.append(x0)
#         edge_x.append(x1)
#         edge_x.append(None)
        
#         edge_y.append(y0)
#         edge_y.append(y1)
#         edge_y.append(None)

edge_x = []
edge_y = []

for idx in tqdm(list(global_optimal_path) + [list(global_optimal_path)[0]]):
    
    x0, y0 = points[idx]
    
    edge_x.append(x0)
    edge_y.append(y0)

In [ ]:


fig = go.Figure(data=go.Scattergl(
    x=edge_x,
    y=edge_y,
    mode="markers+lines",
    # mode="markers"
))
fig.update_layout(height=750)
fig.show()


In [ ]:


fig = go.Figure(data=go.Scattergl(
    x=edge_x,
    y=edge_y,
    mode="markers+lines",
    # mode="markers"
))
fig.update_layout(height=750)
fig.show()


In [ ]:
fig = go.Figure(data=go.Scattergl(
    x=edge_x,
    y=edge_y,
    mode="markers+lines",
    # mode="markers"
))
fig.update_layout(width=1200, height=1200*layout_ratio)
fig.show()

In [ ]:

x_points = [x for x, y in points]
y_points = [y for x, y in points]
layout_ratio = max(x_points)/max(y_points)

fig = go.Figure(data=go.Scattergl(
    x=x_points,
    y=y_points,
    # mode="markers+lines",
    mode="markers"
))
fig.update_layout(width=1200, height=1200*layout_ratio)
fig.show()

In [ ]:

x_points = [x for x, y in exploring_path]
y_points = [y for x, y in exploring_path]
layout_ratio = max(x_points)/max(y_points)

fig = go.Figure(data=go.Scattergl(
    x=x_points,
    y=y_points,
    mode="markers+lines",
    # mode="markers"
))
fig.update_layout(width=1200, height=1200*layout_ratio)
fig.show()

In [ ]:
# for iter in tqdm(range(max_iterations)):
    
#     if temperature < final_temperature:
#         break
        
#     for _ in range(10):
        
#         # pick a random point or pick points carefully
#         alter_point = random.choice(range(total_nodes))
        
#         point_idx1 = optimal_path.index(alter_point)
    
#         # get all the neighbors to that point        
#         sorted_neighbor_points = np.argsort(DISTANCE_MATRIX[alter_point])[1:search_idx]
        
#         # repeatedly do 2-opt for all the neighbors
#         for neighbor_idx in sorted_neighbor_points:
    
#             point_idx2 = optimal_path.index(neighbor_idx)
#             new_path = do_2opt(point_idx1, point_idx2, optimal_path)
#             new_distance = calculate_travel(new_path, points)
        
#             delta = new_distance - optimal_distance
#             acceptance_prob = np.exp(-delta/temperature)
#             to_pick_prob = random.random()
            
    
#             if delta < 0 or to_pick_prob < acceptance_prob:
                
#                 optimal_path = new_path.copy()
#                 optimal_distance = new_distance
                
#                 if new_distance < global_optimal_distance:
#                     global_optimal_path = new_path.copy()
#                     global_optimal_distance = new_distance
#                     global_temperature = temperature
    
#             # else:
#             #     optimal_path = global_optimal_path.copy()
#             #     optimal_distance = global_optimal_distance
                

#     if iter%10 == 0:
#         print(restart_counter, iter, search_idx, round(global_optimal_distance, 4), round(temperature, 4), round(cooling_rate, 5))


#     if round(global_optimal_distance, 1) < last_known_distance:
#         last_known_distance = round(global_optimal_distance, 1)
#         restart_counter = 0
#     else:
#         restart_counter += 1


#     if restart_counter >= 3:
                
#         if cooling_rate*1.001 < 0.995:
#             restart_counter = 0  
#             cooling_rate *= 1.001
#             if global_temperature != init_temperature:
#                 temperature = global_temperature

#         elif iter % 10 == 0 and search_idx > 25:
#             restart_counter = 0
#             search_idx = int(search_idx//1.1)
            
#         temperature *= cooling_rate
   

#     # if iter % 100 == 0 and search_idx > 15:
#     #     search_idx -= 5
        
#     # if restart_counter > 100 and temperature < 0.1:
#     #     print("long due. ending loops")
#     #     break

In [ ]:
# for iter in tqdm(range(max_iterations)):
    
#     if temperature < final_temperature:
#         break
        
#     for _ in range(100000):
        
#         # pick a random point or pick points carefully
#         alter_point1, alter_point2 = random.sample(range(nodeCount), 2)
        
#         # point_idx1 = optimal_path.index(alter_point1)
#         # point_idx2 = optimal_path.index(alter_point2)

#         while tabu_list.is_present(alter_point1) and tabu_list.is_present(alter_point2):
#             alter_point1, alter_point2 = random.sample(range(nodeCount), 2)

#         tabu_list.insert(alter_point1)
#         tabu_list.insert(alter_point2)

#         point_idx1 = np.where(optimal_path==alter_point1)[0][0]
#         point_idx2 = np.where(optimal_path==alter_point2)[0][0]
    
#         new_path = do_2opt(point_idx1, point_idx2, optimal_path)
#         new_distance = calculate_travel(new_path)
    
#         delta = new_distance - optimal_distance
#         acceptance_prob = np.exp(-delta/temperature)
#         to_pick_prob = random.random()
        

#         if delta < 0 or to_pick_prob < acceptance_prob:
            
#             optimal_path = new_path.copy()
#             optimal_distance = new_distance
            
#             if new_distance < global_optimal_distance:
#                 global_optimal_path = new_path.copy()
#                 global_optimal_distance = new_distance
#                 global_temperature = temperature
    
#             # else:
#             #     optimal_path = global_optimal_path.copy()
#             #     optimal_distance = global_optimal_distance
                

#     if iter%10 == 0:
#         print(restart_counter, iter, search_idx, round(global_optimal_distance, 4), round(temperature, 4), round(cooling_rate, 5))


#     if round(global_optimal_distance, 1) < last_known_distance:
#         last_known_distance = round(global_optimal_distance, 1)
#         restart_counter = 0
#     else:
#         restart_counter += 1


#     if restart_counter >= 3:
                
#         if cooling_rate*1.001 < 0.995:
#             restart_counter = 0  
#             cooling_rate *= 1.001
#             if global_temperature != init_temperature:
#                 temperature = global_temperature
            
#         temperature *= cooling_rate
   

#     # if iter % 100 == 0 and search_idx > 15:
#     #     search_idx -= 5
        
#     # if restart_counter > 20:
#     #     print("long due. ending loops")
#     #     break